# Rankings

In [ ]:
ENV="prod"

## Connection to PostgreSQL database

In [ ]:
from itables import init_notebook_mode, show
from ipywidgets import interact, widgets

from postgres.postgres_querying import PostgresQuerying
from src.rankings.teams_ranking import TeamsRanking
from src.rankings.players_ranking import PlayersRanking

init_notebook_mode(all_interactive=True)

In [ ]:
db = PostgresQuerying(ENV)

## Rankings

### Utils

In [ ]:
# TODO: 
#   class Utils qui fournit la liste des championnats, des saisons, ...
#   qui est importée aussi dans les classes de ranking / opposition pour faire les checks et les boucles

In [ ]:
ID_COMP: str = 'ligue_1'
SEASON: str = '2024-2025'
SIDE: str = 'both'
FIRST_WEEK: int = 1
LAST_WEEK: int = 100
FIRST_DATE = '1970-01-01'
LAST_DATE = '2099-12-31'

N: int = 1000000
ROUND: int = 2

### Teams ranking

In [ ]:
teams_ranking = TeamsRanking(db)
t_ranking = teams_ranking.build_ranking("ligue_1", ["2024_2025"], FIRST_WEEK, LAST_WEEK, FIRST_DATE, LAST_DATE, SIDE, N, ROUND, False)

In [ ]:
show(t_ranking.sort_values('Points', ascending=False), paging=False)

In [ ]:
show(t_ranking[['Matches', 'Club', 'Wins', 'Draws', 'Loses', 'Points']].set_index('Club').loc['Rennes'], paging=False)
#, 'xP', 'Diff Points'

### Players Rankings

In [ ]:
players_ranking = PlayersRanking(db)
p_ranking = players_ranking.build_ranking([], ["2023-2024", "2024-2025"], FIRST_WEEK, LAST_WEEK, FIRST_DATE, LAST_DATE, SIDE)

In [ ]:
show(p_ranking.sort_values('Goals', ascending=False), paging=False)

In [ ]:
show(p_ranking[p_ranking['Player'] == 'Amine Gouiri'], paging=False)

In [ ]:
show(p_ranking[p_ranking['Player'] == 'Kylian Mbappé'], paging=False)

## Distinct positions

In [ ]:
all_positions = db.df_from_query(f"select position from season_{SEASON.replace('-', '_')}.player_main_stats ps group by position;")["position"].dropna().tolist()
flattened_positions = []
for sublist in [position.split(',') for position in all_positions]:
    flattened_positions.extend(sublist)

In [ ]:
list(set(all_positions))

In [ ]:
unique_positions = list(set(flattened_positions))
general_positions = ['gk', 'def', 'mid', 'att']

{
    'gk': 'gk',
    'df': 'def',
    'lb': 'def',
    'cb': 'def',
    'wb': 'def', #wide back
    'rb': 'def',
    'dm': 'mid',
    'cm': 'mid',
    'lm': 'mid',
    'rm': 'mid',
    'mf': 'mid', #in-sub players
    'am': 'mid', #attacking midfielder
    'lw': 'att', #winger
    'rw': 'att',
    'fw': 'att'
}

# Closing the PostgreSQL session

In [ ]:
db.close()